In [1]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import tensorflow.keras as keras
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator

from sklearn.model_selection import train_test_split

In [2]:
# Load lines from mfeat-pix.txt
features = open('mfeat-pix.txt').readlines()

# Create labels for each line, 0-9 for each 200 lines
labels = []
for i in range(10):
  for j in range(200):
    labels.append(i)
labels = np.array(labels)

# Convert each line to a numpy array
for i in range(len(features)):
  features[i] = np.array(features[i].split()).astype('float').reshape(16*15)
  # Normalize
  features[i] = features[i] / 6
features = np.array(features)

print('features: {}, labels: {}'.format(features.shape, labels.shape))

features: (2000, 240), labels: (2000,)


In [3]:
X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.2, random_state=40, stratify=labels)

In [6]:
image_generator = ImageDataGenerator(
    rotation_range=20,
    zoom_range=[1, 0.9],
    width_shift_range=0.05,
    height_shift_range=0.05,
    shear_range=20,
    fill_mode="constant",
    cval=0
)
image_generator.fit(X_train.reshape(X_train.shape[0], 16, 15, 1))

In [9]:
X_train_augmented = []
y_train_augmented = []

number_images_per_digit = 840

counters = np.zeros(shape=(10, 1))

flag_leave = False
for X, Y in image_generator.flow(
    features.reshape(features.shape[0], 16, 15, 1),
    labels,
    batch_size=1000,
    shuffle=False,
):
    for i in range(100):
        for j in range(1000):
            img_class = int(Y[j])
            
            if counters[img_class] < number_images_per_digit:
                counters[img_class] += 1
                X_train_augmented.append(X[j].reshape(16, 15))
                y_train_augmented.append(int(Y[j]))
            
            else:
                if (counters >= number_images_per_digit).all():
                    flag_leave = True
                    break
        if flag_leave:
            break
    if flag_leave:
        break
        
X_train_augmented = np.array(X_train_augmented)
y_train_augmented = np.array(y_train_augmented)

# Add original data to augmented data
X_train_augmented = np.concatenate((X_train_augmented.reshape(X_train_augmented.shape[0], 16, 15, 1), 
                                    X_train.reshape(X_train.shape[0], 16, 15, 1)))
y_train_augmented = np.concatenate((y_train_augmented, y_train))

print(X_train_augmented.shape)
print(y_train_augmented.shape)
print(X_test.shape)
print(y_test.shape)

(10000, 16, 15, 1)
(10000,)
(400, 240)
(400,)


In [10]:
print('labels_augmented: ', np.unique(y_train_augmented, return_counts=True))

np.save("X_train_augmented.npy", X_train_augmented)
np.save("y_train_augmented.npy", y_train_augmented)
np.save("X_test_augmented.npy", X_test)
np.save("y_test_augmented.npy", y_test)

labels_augmented:  (array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9]), array([1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000],
      dtype=int64))
